In [1]:
import pandas as pd
import json
from selenium import webdriver
import time

filename = "moexrating-stocks.json"

def getPrices():
    options = webdriver.ChromeOptions()
    chrome_prefs = {}
    options.experimental_options["prefs"] = chrome_prefs
    chrome_prefs["profile.default_content_settings"] = {"images": 2}
    chrome_prefs["profile.managed_default_content_settings"] = {"images": 2}
    options.headless = True
    options.add_argument("--headless")
    options.add_argument('--no-sandbox')
    driver = webdriver.Chrome(options=options)  
    driver.get('https://ru.investing.com/equities/russia')
    driver.find_element_by_xpath('//*[@id="13666"]').click()
    print('waiting data')
    for i in range(1,11):
        try:
            print(f'try {i} load technical data')
            time.sleep(5)
            tbl = driver.find_element_by_xpath('//*[@id="cross_rate_markets_stocks_1"]').get_attribute('outerHTML')
            break
        except:
            print('cannot load technical data')
            if i == 10: return
    driver.close()
    
    return tbl

with open(filename) as file:
    data = json.load(file)

df = pd.read_json("moexrating-stocks.json")
names10 = df['name'].head(10).values.tolist()
print(names10)

tbl = getPrices()

['Сбербанк (прив.)', 'Татнефть', 'Группа ЛСР', 'Сбербанк', 'Интер РАО ЕЭС ОАО', 'ММК', 'Северсталь', 'Татнефть (прив.)', 'Газпром', 'Норникель']
waiting data
try 1 load technical data


In [2]:
from bs4 import BeautifulSoup

datalist = []

soup = BeautifulSoup(tbl, 'lxml')
trs = soup.find_all('tr')
for name10 in names10:
    for tr in trs[1:]:
        tds = tr.find_all('td')
        name = tds[1].find('a').text
        price = float(tds[2].text.replace('.','').replace(',','.'))
        if name == name10:
            datalist.append({'name': name, 'price': price})
    
print(datalist)
    
#     data = {}
#     data['name'] = 

[{'name': 'Сбербанк (прив.)', 'price': 297.14}, {'name': 'Татнефть', 'price': 493.0}, {'name': 'Группа ЛСР', 'price': 725.8}, {'name': 'Сбербанк', 'price': 321.7}, {'name': 'Интер РАО ЕЭС ОАО', 'price': 4.4545}, {'name': 'ММК', 'price': 63.25}, {'name': 'Северсталь', 'price': 1575.0}, {'name': 'Татнефть (прив.)', 'price': 451.3}, {'name': 'Газпром', 'price': 346.78}, {'name': 'Норникель', 'price': 21896.0}]


In [5]:
balance = 10000
for i in range(0,10):
    cost = balance*(0.145-i*0.01)
    quantity = cost/float(datalist[i]['price'])
    datalist[i]['cost'] = cost
    datalist[i]['quantity'] = quantity
print(datalist)
    

[{'name': 'Сбербанк (прив.)', 'price': 297.14, 'cost': 1450.0, 'quantity': 4.879854613986673}, {'name': 'Татнефть', 'price': 493.0, 'cost': 1349.9999999999998, 'quantity': 2.7383367139959427}, {'name': 'Группа ЛСР', 'price': 725.8, 'cost': 1249.9999999999998, 'quantity': 1.7222375310002753}, {'name': 'Сбербанк', 'price': 321.7, 'cost': 1150.0, 'quantity': 3.574759092322039}, {'name': 'Интер РАО ЕЭС ОАО', 'price': 4.4545, 'cost': 1049.9999999999998, 'quantity': 235.71669098664265}, {'name': 'ММК', 'price': 63.25, 'cost': 949.9999999999999, 'quantity': 15.019762845849801}, {'name': 'Северсталь', 'price': 1575.0, 'cost': 849.9999999999999, 'quantity': 0.5396825396825397}, {'name': 'Татнефть (прив.)', 'price': 451.3, 'cost': 749.9999999999999, 'quantity': 1.6618657212497228}, {'name': 'Газпром', 'price': 346.78, 'cost': 649.9999999999999, 'quantity': 1.874387219562835}, {'name': 'Норникель', 'price': 21896.0, 'cost': 549.9999999999999, 'quantity': 0.02511874314943368}]
